In [59]:
#load "sig.fsx"
open CSCI374.ExtraReflection

# F# Programming - Recursion

## Example:

- Implement an exponential function, $x^y = x * \cdots * x$

In [38]:
// calculate power (imperative way)
let pow x y = 
    let mutable res = 1
    let mutable count = y
    while count > 0 do
        res <- res * x 
        count <- count - 1
    res
sgn pow    

Function: x:int -> y:int -> int

In [39]:
pow 2 10

1024

**Recursion** means defining a function in terms of itself; in other words, the function calls itself within its definition. 

- Recursion is often used in functional programming where you would use a loop in imperative programming.

- Many believe that algorithms are much easier to understand when expressed in terms of recursion rather than loops.

- “Makes sense” because calls to same function solve “simpler” problems

In [48]:
let rec pow_rec x y = 
    if y = 0 then 1
    else x * pow_rec x (y-1)
        
sgn pow_rec

Function: x:int -> y:int -> int

In [49]:
pow_rec 2 11

2048

## Fibonacci numbers

$$F_0 = 0, F_1 = 1$$
$$F_{n} = F_{n-1} + F_{n-2}$$


In [50]:
let rec fib n = 
    if n = 0 then 0
    else if n = 1 then 1
    else (fib (n-1)) + (fib (n-2))
                
sgn fib

Function: n:int -> int

In [51]:
fib 10, fib 11, fib 12

Item1,Item2,Item3
55,89,144


Recursion more powerful than loops

- We won’t use a single loop in F#
- Loops often (not always) obscure simple, elegant solutions

In [57]:
let lst = 1 :: []
lst, sgn (lst)

Item1,Item2
[ 1 ],Value: list<int>


In [56]:
1 :: [2; 3]

index,value
0,1
1,2
2,3


In [58]:
1 :: (2 :: ( 3 :: [])) // = [1;2;3]

index,value
0,1
1,2
2,3


In [52]:
let l = [1; 2; 3; 4; 5]
l

index,value
0,1
1,2
2,3
3,4
4,5


In [60]:
let rec sum_list (xs : int list) = 
    if List.isEmpty xs
    then 0
    else List.head xs + sum_list (List.tail xs)

sgn sum_list

Function: xs:list<int> -> int

In [61]:
sum_list [1 .. 100]

5050

In [62]:
let rec sum_list (xs : int list) = 
    if xs.IsEmpty
    then 0
    else xs.Head + sum_list xs.Tail

sgn sum_list

Function: xs:list<int> -> int

In [64]:
sum_list [10 .. 100]

5005

In [65]:
let rec countdown (x : int) =
    if x = 0
    then []
    else x :: countdown (x-1)
    
sgn countdown

Function: x:int -> list<int>

In [66]:
countdown 11

index,value
0,11
1,10
2,9
3,8
4,7
5,6
6,5
7,4
8,3
9,2


In [67]:
let rec append (xs : int list) (ys : int list) =
    if xs.IsEmpty 
    then ys
    else xs.Head :: append xs.Tail ys
    
sgn append

Function: xs:list<int> -> ys:list<int> -> list<int>

In [68]:
append [1; 2; 3 ; 4] (countdown 4)

index,value
0,1
1,2
2,3
3,4
4,4
5,3
6,2
7,1


### Recursion again

Functions over lists are usually recursive
- Only way to "get to all the elements"
    - What should the answer be for the empty list?
    - What should the answer be for a non-empty list?
- Typically in terms of the answer for the tail of the list!

Similarly, functions that produce lists of potentially any size will be recursive
- You create a list out of smaller lists

### Avoid repeated recursion

Consider this code and the recursive calls it makes
- Don't worry about calls to `isEmpty`, `head`, and `tail` because they do a small constant amount of work

In [69]:
let rec bad_max (xs : int list) =
    if xs.IsEmpty
    then 0 // horrible style; fix later
    else if xs.Tail.IsEmpty
    then xs.Head 
    else if xs.Head > bad_max xs.Tail
    then xs.Head
    else bad_max xs.Tail

sgn bad_max

Function: xs:list<int> -> int

In [79]:
#!time
bad_max [30 .. -1 .. 1]

30

Wall time: 9.1807ms

In [80]:
#!time
bad_max [1 .. 30]

30

Wall time: 9857.3286ms

![bad_rec](img/bad_rec.png)

### Math never lies

Suppose one bad_max call's `if-then-else logic` and calls to `head`, `isEmpty`, `tail` take $10^{-7}$ seconds

- Then bad_max [50,49,...,1] takes 50 x $10^{-7}$ seconds
- And bad_max [1,2,...,50] takes 1.12 x $10^8$ seconds (over 3.5 years)
    - bad_max [1,2,...,55]takes over 1 century
    - Buying a faster computer won’t help much 
    
The key is not to do repeated work that might do repeated work that might do...
- Saving recursive results in local bindings is essential...

![good_rec](img/good_rec.png)

In [81]:
let rec good_max (xs : int list) =
    if xs.IsEmpty
    then 0 // horrible style; fix later
    else if xs.Tail.IsEmpty
    then xs.Head
    else
        let tail_ans = good_max xs.Tail
        if xs.Head > tail_ans
        then xs.Head
        else tail_ans
sgn good_max

Function: xs:list<int> -> int

In [82]:
#!time
good_max [1 .. 55]

55

Wall time: 7.0606ms